In [ ]:
# ==============================================================================
# POPULATION GROWTH PREDICTION FOR HARRIS COUNTY CENSUS TRACTS
# ==============================================================================
# GOAL: Predict population growth rate (% change 2010→2021) for each census tract
# TYPE: Regression Problem
# OUTPUT: Continuous value representing % population change
# ==============================================================================

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings("ignore")

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

## 1. Load and Inspect Data

Load the prepared CSV file that contains:

- Census tract identifiers
- Population data (2010, 2015, 2021)
- Flood risk metrics (% in AE zone, % in X zone, etc.)
- Elevation features (mean, min, max, slope, REM)
- Water proximity (distance to nearest water body)
- Flow accumulation metrics
- Spatial features


In [ ]:
# Load the prepared dataset
# This CSV should be created from your data processing script (01_data_exploration.ipynb)
df = pd.read_csv("../data/processed/census_tracts_features.csv")

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

print(f"\nColumn names:")
print(df.columns.tolist())

print(f"\nData types:")
print(df.dtypes)

print(f"\nMissing values:")
print(df.isnull().sum())

print(f"\nBasic statistics:")
print(df.describe())

## 2. Create Target Variable (Y)

**Target**: % Population Change from 2010 to 2021

Formula: `((pop_2021 - pop_2010) / pop_2010) * 100`

This represents the percentage growth or decline over the ~11 year period.


In [ ]:
# Calculate population change percentage
# Assumes columns: pop_2010, pop_2021 exist in the dataframe
df["pop_change_pct"] = ((df["pop_2021"] - df["pop_2010"]) / df["pop_2010"]) * 100

# Handle edge cases
# Remove tracts with zero population in 2010 (division by zero)
df = df[df["pop_2010"] > 0].copy()

# Remove extreme outliers (optional - adjust thresholds as needed)
# This removes tracts with unrealistic growth (e.g., >500% or <-99%)
df = df[(df["pop_change_pct"] > -99) & (df["pop_change_pct"] < 500)].copy()

print(f"Dataset shape after cleaning: {df.shape}")
print(f"\nTarget variable statistics:")
print(df["pop_change_pct"].describe())

# Visualize distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df["pop_change_pct"], bins=50, edgecolor="black", alpha=0.7)
axes[0].set_xlabel("Population Change (%)")
axes[0].set_ylabel("Frequency")
axes[0].set_title("Distribution of Population Change 2010-2021")
axes[0].axvline(0, color="red", linestyle="--", label="No Change")
axes[0].legend()

axes[1].boxplot(df["pop_change_pct"])
axes[1].set_ylabel("Population Change (%)")
axes[1].set_title("Boxplot of Population Change")
axes[1].axhline(0, color="red", linestyle="--")

plt.tight_layout()
plt.show()

print(f"\nGrowth categories:")
print(f"  Declining (< -5%): {len(df[df['pop_change_pct'] < -5])} tracts")
print(
    f"  Stable (-5% to 5%): {len(df[(df['pop_change_pct'] >= -5) & (df['pop_change_pct'] <= 5)])} tracts"
)
print(f"  Growing (> 5%): {len(df[df['pop_change_pct'] > 5])} tracts")

## 3. Define Feature Sets (X)

**Feature Categories:**

1. **Flood Risk Features:**

   - `pct_in_AE`: % of tract in high-risk AE flood zone
   - `pct_in_X`: % of tract in low-risk X zone (outside 500-year floodplain)
   - `pct_in_VE`: % in coastal high velocity zone (if applicable)
   - `pct_in_floodway`: % in regulatory floodway

2. **Elevation & Topography:**

   - `elev_mean`: Mean elevation (meters)
   - `elev_min`: Minimum elevation
   - `elev_max`: Maximum elevation
   - `elev_std`: Elevation standard deviation (terrain roughness)
   - `slope_mean`: Mean terrain slope
   - `slope_max`: Maximum slope
   - `pct_low_slope`: % of area with slope < 2 degrees (flat/flood-prone)

3. **Hydrological:**

   - `dist_water_m`: Distance to nearest water body (meters)
   - `flowacc_mean`: Mean flow accumulation (drainage potential)
   - `rem_mean`: Relative Elevation Model value (height above nearest drainage)

4. **Baseline Demographics:**

   - `pop_2010`: Initial population (baseline)
   - `pop_density_2010`: Population density in 2010 (people/sq km)
   - `area_sq_km`: Tract area

5. **Spatial Features (optional):**
   - `centroid_lat`: Latitude of tract centroid
   - `centroid_lon`: Longitude of tract centroid
   - `dist_to_downtown_km`: Distance to downtown Houston


In [ ]:
# Define feature columns
# Adjust these based on what's actually in your processed CSV
feature_cols = [
    # Flood risk
    "pct_in_AE",
    "pct_in_X",
    "pct_in_floodway",
    # Elevation & topography
    "elev_mean",
    "elev_min",
    "elev_std",
    "slope_mean",
    "slope_max",
    "pct_low_slope",
    # Hydrological
    "dist_water_m",
    "flowacc_mean",
    "rem_mean",
    # Baseline demographics
    "pop_2010",
    "pop_density_2010",
    "area_sq_km",
    # Spatial (optional)
    # 'centroid_lat',
    # 'centroid_lon',
    # 'dist_to_downtown_km'
]

# Check which features are actually available
available_features = [col for col in feature_cols if col in df.columns]
missing_features = [col for col in feature_cols if col not in df.columns]

print(f"Available features ({len(available_features)}): {available_features}")
print(f"\nMissing features ({len(missing_features)}): {missing_features}")

# Use only available features
X = df[available_features].copy()
y = df["pop_change_pct"].copy()

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")

# Check for missing values in features
print(f"\nMissing values in features:")
print(X.isnull().sum())

# Handle missing values (if any)
# Option 1: Drop rows with any missing values
# X = X.dropna()
# y = y[X.index]

# Option 2: Fill with median (more common for ML)
if X.isnull().sum().sum() > 0:
    print("\nFilling missing values with column medians...")
    X = X.fillna(X.median())

## 4. Exploratory Data Analysis (EDA)

Understand relationships between features and target variable.


In [ ]:
# Correlation matrix
plt.figure(figsize=(14, 10))
correlation_data = pd.concat([X, y], axis=1)
corr_matrix = correlation_data.corr()

sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    square=True,
    linewidths=0.5,
)
plt.title("Feature Correlation Matrix", fontsize=14, pad=20)
plt.tight_layout()
plt.show()

# Show correlations with target variable
print("\nCorrelation with population change (sorted by absolute value):")
target_corr = (
    corr_matrix["pop_change_pct"]
    .drop("pop_change_pct")
    .abs()
    .sort_values(ascending=False)
)
print(target_corr)

In [ ]:
# Scatter plots of top correlated features
top_features = target_corr.head(6).index.tolist()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(top_features):
    axes[i].scatter(X[feature], y, alpha=0.5, s=10)
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel("Population Change (%)")
    axes[i].set_title(f"{feature} vs Pop Change")

    # Add trend line
    z = np.polyfit(X[feature], y, 1)
    p = np.poly1d(z)
    axes[i].plot(
        X[feature].sort_values(),
        p(X[feature].sort_values()),
        "r--",
        linewidth=2,
        alpha=0.7,
    )

plt.tight_layout()
plt.show()

## 5. Data Preparation & Train-Test Split

Split data into training (75%) and testing (25%) sets.


In [ ]:
# Train-test split (75-25)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

print(f"Training set size: {X_train.shape[0]} tracts")
print(f"Testing set size: {X_test.shape[0]} tracts")
print(f"Number of features: {X_train.shape[1]}")

# Feature scaling (important for some algorithms)
# Tree-based models don't require scaling, but linear models do
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nData prepared for modeling!")

## 6. Model Training & Comparison

Train multiple regression models and compare performance:

- Random Forest (baseline)
- Gradient Boosting
- Ridge Regression
- Lasso Regression


In [ ]:
# Initialize models
models = {
    "Random Forest": RandomForestRegressor(
        n_estimators=200, max_depth=10, min_samples_split=10, random_state=42
    ),
    "Gradient Boosting": GradientBoostingRegressor(
        n_estimators=300, learning_rate=0.05, max_depth=5, random_state=42
    ),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
}

# Store results
results = {}

print("Training models...\n")

for name, model in models.items():
    print(f"Training {name}...")

    # Use scaled data for linear models, original for tree-based
    if name in ["Ridge", "Lasso"]:
        model.fit(X_train_scaled, y_train)
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

    results[name] = {
        "model": model,
        "train_r2": train_r2,
        "test_r2": test_r2,
        "train_mae": train_mae,
        "test_mae": test_mae,
        "test_rmse": test_rmse,
        "predictions": y_pred_test,
    }

    print(f"  Train R²: {train_r2:.4f} | Test R²: {test_r2:.4f}")
    print(f"  Train MAE: {train_mae:.2f} | Test MAE: {test_mae:.2f}")
    print(f"  Test RMSE: {test_rmse:.2f}\n")

print("✅ All models trained!")

In [ ]:
# Compare model performance
comparison_df = pd.DataFrame(
    {
        "Model": list(results.keys()),
        "Train R²": [results[m]["train_r2"] for m in results.keys()],
        "Test R²": [results[m]["test_r2"] for m in results.keys()],
        "Test MAE": [results[m]["test_mae"] for m in results.keys()],
        "Test RMSE": [results[m]["test_rmse"] for m in results.keys()],
    }
)

comparison_df = comparison_df.sort_values("Test R²", ascending=False)
print("\n" + "=" * 70)
print("MODEL COMPARISON")
print("=" * 70)
print(comparison_df.to_string(index=False))
print("=" * 70)

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# R² scores
comparison_df.plot(x="Model", y=["Train R²", "Test R²"], kind="bar", ax=axes[0])
axes[0].set_ylabel("R² Score")
axes[0].set_title("Model R² Comparison (Higher is Better)")
axes[0].set_ylim(0, 1)
axes[0].legend(["Training", "Testing"])
axes[0].axhline(0.7, color="green", linestyle="--", alpha=0.5, label="Good threshold")

# MAE scores
comparison_df.plot(x="Model", y="Test MAE", kind="bar", ax=axes[1], color="coral")
axes[1].set_ylabel("Mean Absolute Error (%)")
axes[1].set_title("Model MAE Comparison (Lower is Better)")
axes[1].legend(["Test MAE"])

plt.tight_layout()
plt.show()

## 7. Select Best Model & Analyze Results


In [ ]:
# Select best model based on Test R²
best_model_name = comparison_df.iloc[0]["Model"]
best_model_info = results[best_model_name]
best_model = best_model_info["model"]

print(f"✅ Best Model: {best_model_name}")
print(f"   Test R²: {best_model_info['test_r2']:.4f}")
print(f"   Test MAE: {best_model_info['test_mae']:.2f}%")
print(f"   Test RMSE: {best_model_info['test_rmse']:.2f}%")

# Predicted vs Actual scatter plot
y_pred_best = best_model_info["predictions"]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Scatter plot
axes[0].scatter(y_test, y_pred_best, alpha=0.5, s=30)
axes[0].plot(
    [y_test.min(), y_test.max()],
    [y_test.min(), y_test.max()],
    "r--",
    lw=2,
    label="Perfect Prediction",
)
axes[0].set_xlabel("Actual Population Change (%)")
axes[0].set_ylabel("Predicted Population Change (%)")
axes[0].set_title(
    f'{best_model_name}: Predicted vs Actual\nR² = {best_model_info["test_r2"]:.3f}'
)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals plot
residuals = y_test - y_pred_best
axes[1].scatter(y_pred_best, residuals, alpha=0.5, s=30)
axes[1].axhline(0, color="red", linestyle="--", lw=2)
axes[1].set_xlabel("Predicted Population Change (%)")
axes[1].set_ylabel("Residuals (Actual - Predicted)")
axes[1].set_title(f"{best_model_name}: Residuals Plot")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Distribution of residuals
plt.figure(figsize=(10, 5))
plt.hist(residuals, bins=50, edgecolor="black", alpha=0.7)
plt.axvline(0, color="red", linestyle="--", linewidth=2)
plt.xlabel("Residuals (%)")
plt.ylabel("Frequency")
plt.title(f"Distribution of Prediction Errors - {best_model_name}")
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nResidual Statistics:")
print(f"  Mean: {residuals.mean():.2f}%")
print(f"  Std: {residuals.std():.2f}%")
print(f"  Median: {residuals.median():.2f}%")

## 8. Feature Importance Analysis

Understand which features most influence population growth predictions.


In [ ]:
# Feature importance (only for tree-based models)
if best_model_name in ["Random Forest", "Gradient Boosting"]:
    importance = pd.DataFrame(
        {"Feature": available_features, "Importance": best_model.feature_importances_}
    ).sort_values("Importance", ascending=False)

    print("\nFeature Importance (Top 10):")
    print(importance.head(10).to_string(index=False))

    # Plot
    plt.figure(figsize=(10, 8))
    top_n = 15
    importance_top = importance.head(top_n).sort_values("Importance", ascending=True)

    plt.barh(importance_top["Feature"], importance_top["Importance"])
    plt.xlabel("Importance Score")
    plt.ylabel("Feature")
    plt.title(f"Top {top_n} Most Important Features - {best_model_name}")
    plt.tight_layout()
    plt.show()

else:
    # For linear models, show coefficients
    coef_df = pd.DataFrame(
        {"Feature": available_features, "Coefficient": best_model.coef_}
    )
    coef_df["Abs_Coefficient"] = coef_df["Coefficient"].abs()
    coef_df = coef_df.sort_values("Abs_Coefficient", ascending=False)

    print("\nFeature Coefficients (Top 10 by absolute value):")
    print(coef_df[["Feature", "Coefficient"]].head(10).to_string(index=False))

    # Plot
    plt.figure(figsize=(10, 8))
    top_n = 15
    coef_top = coef_df.head(top_n).sort_values("Coefficient", ascending=True)

    colors = ["red" if x < 0 else "green" for x in coef_top["Coefficient"]]
    plt.barh(coef_top["Feature"], coef_top["Coefficient"], color=colors, alpha=0.7)
    plt.xlabel("Coefficient Value")
    plt.ylabel("Feature")
    plt.title(
        f"Top {top_n} Feature Coefficients - {best_model_name}\n(Red=Negative, Green=Positive)"
    )
    plt.axvline(0, color="black", linestyle="--", linewidth=1)
    plt.tight_layout()
    plt.show()

## 9. Hyperparameter Tuning (Optional)

Fine-tune the best model using Grid Search Cross-Validation.


In [ ]:
# Hyperparameter tuning for best model
# WARNING: This can take several minutes!

if best_model_name == "Random Forest":
    param_grid = {
        "n_estimators": [100, 200, 300],
        "max_depth": [8, 10, 12],
        "min_samples_split": [5, 10, 15],
        "min_samples_leaf": [2, 4],
    }
    base_model = RandomForestRegressor(random_state=42)

elif best_model_name == "Gradient Boosting":
    param_grid = {
        "n_estimators": [200, 300, 400],
        "learning_rate": [0.01, 0.05, 0.1],
        "max_depth": [3, 5, 7],
        "min_samples_split": [5, 10],
    }
    base_model = GradientBoostingRegressor(random_state=42)

else:
    print(f"Skipping grid search for {best_model_name}")
    base_model = None

if base_model is not None:
    print(f"Starting Grid Search for {best_model_name}...")
    print(f"Testing {np.prod([len(v) for v in param_grid.values()])} combinations...")

    grid_search = GridSearchCV(
        base_model, param_grid, cv=5, scoring="r2", n_jobs=-1, verbose=1
    )

    # Use scaled or unscaled data based on model type
    if best_model_name in ["Ridge", "Lasso"]:
        grid_search.fit(X_train_scaled, y_train)
        y_pred_tuned = grid_search.predict(X_test_scaled)
    else:
        grid_search.fit(X_train, y_train)
        y_pred_tuned = grid_search.predict(X_test)

    print(f"\n✅ Grid Search Complete!")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV R²: {grid_search.best_score_:.4f}")

    # Evaluate tuned model
    tuned_r2 = r2_score(y_test, y_pred_tuned)
    tuned_mae = mean_absolute_error(y_test, y_pred_tuned)
    tuned_rmse = np.sqrt(mean_squared_error(y_test, y_pred_tuned))

    print(f"\nTuned Model Performance:")
    print(f"  Test R²: {tuned_r2:.4f} (Before: {best_model_info['test_r2']:.4f})")
    print(f"  Test MAE: {tuned_mae:.2f}% (Before: {best_model_info['test_mae']:.2f}%)")
    print(
        f"  Test RMSE: {tuned_rmse:.2f}% (Before: {best_model_info['test_rmse']:.2f}%)"
    )

    # Update best model if improved
    if tuned_r2 > best_model_info["test_r2"]:
        print("\n✅ Tuned model is better! Updating...")
        best_model = grid_search.best_estimator_
        best_model_info["test_r2"] = tuned_r2
        best_model_info["test_mae"] = tuned_mae
        best_model_info["test_rmse"] = tuned_rmse
    else:
        print("\nOriginal model performs better. Keeping original.")

## 10. Make Predictions for Future Growth (2030/2040)

Use the trained model to predict future population changes for all census tracts.


In [ ]:
# Make predictions for all census tracts
if best_model_name in ["Ridge", "Lasso"]:
    df["predicted_growth_pct"] = best_model.predict(
        scaler.transform(df[available_features])
    )
else:
    df["predicted_growth_pct"] = best_model.predict(df[available_features])

# Calculate predicted 2030 population (assuming linear growth from 2021)
# This is a simple projection - adjust years as needed
years_from_baseline = 9  # 2021 to 2030
growth_factor = df["predicted_growth_pct"] / 100  # Convert % to decimal
df["predicted_pop_2030"] = df["pop_2021"] * (
    1 + growth_factor * (years_from_baseline / 11)
)

# Calculate predicted 2040 population
years_from_baseline_2040 = 19  # 2021 to 2040
df["predicted_pop_2040"] = df["pop_2021"] * (
    1 + growth_factor * (years_from_baseline_2040 / 11)
)

print("Predictions generated for all census tracts!")
print(f"\nSample predictions:")
print(
    df[
        [
            "pop_2010",
            "pop_2021",
            "predicted_growth_pct",
            "predicted_pop_2030",
            "predicted_pop_2040",
        ]
    ].head(10)
)

# Summary statistics
print("\nPredicted Growth Distribution:")
print(f"  Mean growth: {df['predicted_growth_pct'].mean():.2f}%")
print(f"  Median growth: {df['predicted_growth_pct'].median():.2f}%")
print(f"  Std dev: {df['predicted_growth_pct'].std():.2f}%")
print(f"  Min: {df['predicted_growth_pct'].min():.2f}%")
print(f"  Max: {df['predicted_growth_pct'].max():.2f}%")

# Categorize predictions
df["growth_category"] = pd.cut(
    df["predicted_growth_pct"],
    bins=[-np.inf, -5, 5, 15, np.inf],
    labels=["Declining", "Stable", "Growing", "Rapid Growth"],
)

print(f"\nPredicted Growth Categories:")
print(df["growth_category"].value_counts())

## 11. Visualize Predictions


In [ ]:
# Population projections over time
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Total population projection
total_2010 = df["pop_2010"].sum()
total_2021 = df["pop_2021"].sum()
total_2030 = df["predicted_pop_2030"].sum()
total_2040 = df["predicted_pop_2040"].sum()

years = [2010, 2021, 2030, 2040]
totals = [total_2010, total_2021, total_2030, total_2040]

axes[0].plot(years, totals, marker="o", linewidth=2, markersize=10, color="steelblue")
axes[0].scatter(
    [2010, 2021],
    [total_2010, total_2021],
    s=100,
    color="green",
    label="Actual",
    zorder=5,
)
axes[0].scatter(
    [2030, 2040],
    [total_2030, total_2040],
    s=100,
    color="orange",
    label="Predicted",
    zorder=5,
)
axes[0].set_xlabel("Year")
axes[0].set_ylabel("Total Population")
axes[0].set_title("Harris County Total Population Projection")
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].ticklabel_format(style="plain", axis="y")

# Growth rate comparison
axes[1].scatter(df["pop_change_pct"], df["predicted_growth_pct"], alpha=0.5, s=30)
axes[1].plot(
    [df["pop_change_pct"].min(), df["pop_change_pct"].max()],
    [df["pop_change_pct"].min(), df["pop_change_pct"].max()],
    "r--",
    lw=2,
    label="1:1 Line",
)
axes[1].set_xlabel("Actual Growth 2010-2021 (%)")
axes[1].set_ylabel("Predicted Growth (%)")
axes[1].set_title("Actual vs Predicted Growth Patterns")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Harris County Population Projections:")
print(f"  2010 (actual): {total_2010:,.0f}")
print(f"  2021 (actual): {total_2021:,.0f}")
print(f"  2030 (predicted): {total_2030:,.0f}")
print(f"  2040 (predicted): {total_2040:,.0f}")
print(f"\n  Change 2010-2021: {((total_2021-total_2010)/total_2010*100):.1f}%")
print(f"  Predicted change 2021-2030: {((total_2030-total_2021)/total_2021*100):.1f}%")
print(f"  Predicted change 2021-2040: {((total_2040-total_2021)/total_2021*100):.1f}%")

## 12. Export Results

In [ ]:
# Save predictions to CSV
output_cols = [
    "tract_id",
    "pop_2010",
    "pop_2021",
    "pop_change_pct",
    "predicted_growth_pct",
    "predicted_pop_2030",
    "predicted_pop_2040",
    "growth_category",
    "pct_in_AE",
    "elev_mean",
    "dist_water_m",
]

# Adjust columns based on what's actually in your dataframe
available_output_cols = [col for col in output_cols if col in df.columns]

df[available_output_cols].to_csv(
    "../data/processed/census_predictions.csv", index=False
)
print("✅ Predictions saved to: ../data/processed/census_predictions.csv")

# Save model performance metrics
metrics_summary = {
    "Best Model": best_model_name,
    "Test R²": best_model_info["test_r2"],
    "Test MAE (%)": best_model_info["test_mae"],
    "Test RMSE (%)": best_model_info["test_rmse"],
    "Number of Features": len(available_features),
    "Training Size": len(X_train),
    "Test Size": len(X_test),
    "Total Tracts": len(df),
}

metrics_df = pd.DataFrame([metrics_summary])
metrics_df.to_csv("../data/processed/model_performance.csv", index=False)
print("✅ Model metrics saved to: ../data/processed/model_performance.csv")

# Optional: Save the model for future use
# import joblib
# joblib.dump(best_model, '../models/population_growth_model.pkl')
# joblib.dump(scaler, '../models/scaler.pkl')
# print("✅ Model saved to: ../models/population_growth_model.pkl")

## 13. Key Findings Summary

**Regression Problem**: Predict % population change for Harris County census tracts

**Target Variable (Y)**:

- `pop_change_pct` = ((pop_2021 - pop_2010) / pop_2010) × 100

**Key Features (X)**:

- Flood risk metrics (% in AE zone, X zone, etc.)
- Elevation & topography (mean, slope, terrain roughness)
- Hydrological features (distance to water, flow accumulation, REM)
- Baseline demographics (2010 population, density)

**Model Performance**:

- Algorithm used will be displayed after running Section 6
- Test R² score indicates model's predictive accuracy
- MAE shows average prediction error in percentage points

**Next Steps**:

1. Run this notebook with your processed CSV data
2. Analyze which features most influence growth
3. Use predictions to identify high-growth and declining areas
4. Overlay with flood risk maps for planning insights
5. Consider climate scenarios and infrastructure changes for future refinement


In [ ]:
# Histogram of predicted growth
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(
    df["predicted_growth_pct"], bins=50, edgecolor="black", alpha=0.7, color="steelblue"
)
plt.axvline(0, color="red", linestyle="--", linewidth=2, label="No Change")
plt.axvline(
    df["predicted_growth_pct"].mean(),
    color="green",
    linestyle="--",
    linewidth=2,
    label=f'Mean: {df["predicted_growth_pct"].mean():.1f}%',
)
plt.xlabel("Predicted Population Growth (%)")
plt.ylabel("Number of Census Tracts")
plt.title("Distribution of Predicted Growth Rates")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
growth_counts = df["growth_category"].value_counts()
colors_cat = ["red", "yellow", "lightgreen", "darkgreen"]
plt.bar(
    growth_counts.index,
    growth_counts.values,
    color=colors_cat,
    alpha=0.7,
    edgecolor="black",
)
plt.ylabel("Number of Census Tracts")
plt.title("Census Tracts by Growth Category")
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

In [ ]:
# Distribution of predicted growth
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(
    df["predicted_growth_pct"], bins=50, edgecolor="black", alpha=0.7, color="steelblue"
)
axes[0].axvline(
    df["predicted_growth_pct"].mean(),
    color="red",
    linestyle="--",
    linewidth=2,
    label=f'Mean: {df["predicted_growth_pct"].mean():.1f}%',
)
axes[0].axvline(0, color="black", linestyle="-", linewidth=1, alpha=0.5)
axes[0].set_xlabel("Predicted Growth (%)")
axes[0].set_ylabel("Number of Census Tracts")
axes[0].set_title("Distribution of Predicted Population Growth")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Growth categories
df["growth_category"].value_counts().plot(kind="bar", ax=axes[1], color="coral")
axes[1].set_xlabel("Growth Category")
axes[1].set_ylabel("Number of Census Tracts")
axes[1].set_title("Census Tracts by Predicted Growth Category")
axes[1].tick_params(axis="x", rotation=45)
axes[1].grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()